In [1]:
import $file.hw9stdlib
import hw9stdlib._

Compiling /home/user/Downloads/Assignment9/hw9stdlib.sc

import $file.$        

import hw9stdlib._

In [2]:
type Parser[S,D] = List[S] => List[(D, List[S])]

def char(c : Char) : Parser[Char,Char] = 
(ss : List[Char]) => ss match {
    case Empty      => Empty
    case Cons(s,ss) => char_eq(s,c) match {
        case True  => singleton((s, ss))
        case False => Empty
    }
}

def success[S, D](x : D) : Parser[S, D] = 
  (ss : List[S]) => singleton((x, ss))

def failure[S, D]() : Parser[S,D] = (ss : List[S]) => Empty

def choose[S, D](p : Parser[S, D], q : Parser[S, D]) : Parser[S, D] = (ss : List[S]) =>
{
    val p_res = p(ss)
    val q_res = q(ss)
    append(p_res, q_res)
} 

def runParser[D](p : Parser[Char, D], s : String) : Maybe[D] = p(string_to_list(s)) match {
    case Empty                   => None
    case Cons((x, Empty), Empty) => Just(x)
    case _                       => None
}

def satisfies[S](p : (S => Bool)) : Parser[S, S] = (xs : List[S]) => xs match {
    case Empty       => Empty
    case Cons(x, xs) => p(x) match{
        case True  => singleton((x,xs))
        case False => Empty
    } 
}

def bind[S, D, E](p : Parser[S,D], q : (D => Parser[S,E]) ) : Parser[S,E] = 
  (ss : List[S]) => {
      val join = (res : (D, List[S])) => res match {case (d, ss2) => q(d)(ss2)}
      concatMap(join, p(ss))
  } 

def stringL(ss : List[Char]) : Parser[Char, List[Char]] = ss match {
    case Empty      => success(Empty)
    case Cons(s,ss) =>        bind(char(s), 
         (c : Char)        => bind(stringL(ss),
         (cs : List[Char]) => success(Cons(c,cs)) ))
}

def string(str : String) : Parser[Char, List[Char]] = stringL(string_to_list(str))

def mapParser[S,A,B](f : (A => B), p : Parser[S,A]) : Parser[S,B] = bind(p, (result : A) => success(f(result)))

def option[S, D](p : Parser[S, D], q : Parser[S, D]) : Parser[S, D] = (ss : List[S]) => 
  p(ss) match {
      case Empty => q(ss)
      case res   => res
  }

def whitespaceChar : Parser[Char,Char] = choose(char(' '),
                                         choose(char('\n'), 
                                         choose(char('\r'), 
                                                char('\t'))))

def many[S,D](p : Parser[S,D]) : Parser[S, List[D]] = option(
                                                        bind(p, 
                                                            (x : D) => bind(many(p),
                                                            (xs : List[D]) => success(Cons(x,xs))))
                                                       , success(Empty)
                                                       )

// A whitespace parser which requires at least one whitespace character
// This is used to parse spaces between lexical elements
def whitespace : Parser[Char, List[Char]] = bind(whitespaceChar, 
                                                 (x : Char)  => bind(many(whitespaceChar),
                                                 (xs : List[Char]) => success(Cons(x,xs))))

// A whitespace parser for zero or more spaces
def maybeWhitespace : Parser[Char, List[Char]] = many(whitespaceChar)

def digit : Parser[Char, Char] = option(char('0'),
                                 option(char('1'),
                                 option(char('2'),
                                 option(char('3'),
                                 option(char('4'),
                                 option(char('5'),
                                 option(char('6'),
                                 option(char('7'),
                                 option(char('8'),
                                        char('9'))))))))))

def numString : Parser[Char, List[Char]] = many(digit)

def digitToNat(c : Char) : Nat = c match {
    case '0' => Zero
    case '1' => one
    case '2' => two
    case '3' => three
    case '4' => four
    case '5' => five
    case '6' => six
    case '7' => seven
    case '8' => eight
    case '9' => nine
}

def stringToNat(xs : List[Char]) : Nat = 
  fold( (x : Nat, acc : Nat) => nat_plus(x, nat_mult(acc, ten)) ,Zero, map(digitToNat, xs))

def number : Parser[Char, Nat] = bind(numString,
                                     (ss : List[Char]) => ss match {
                                         case Empty       => failure()
                                         case Cons(x, xs) => success(stringToNat(ss))
                                     })

defined type Parser
defined function char
defined function success
defined function failure
defined function choose
defined function runParser
defined function satisfies
defined function bind
defined function stringL
defined function string
defined function mapParser
defined function option
defined function whitespaceChar
defined function many
defined function whitespace
defined function maybeWhitespace
defined function digit
defined function numString
defined function digitToNat
defined function stringToNat
defined function number

# Homework 9 - The Lettuce Parser

In the next 2 homeworks we will write a complete parser for Lettuce and then we will have a complete programming language.

The development process for the parser will go something like this:

1. Part A - Develop a lexer for Lettuce
2. Part B - Write the parser using the lexical grammar
3. Part C - Hook it together with the lettuce evaluation function to have a complete interpreter



 ## Part A: The Lexer

We will start by writing a lexer for lettuce. Let's begin by defining the Lexical Grammar for the language:

$$
\begin{align}
    \textbf{Lex} ::=&\ \text{Keyword}\ \textbf{Keyword}\\
    \mid&\ \text{Literal}\ \textbf{Literal}\\
    \mid &\ \text{Identifier}\ \textbf{String}\\
    \mid &\ \text{Operator}\ \textbf{String}\\
    \mid&\ \text{OpenParen}\\
    \mid&\ \text{CloseParen}
\end{align}
$$

$$
\begin{align}
    \textbf{Keyword} ::=&\ \text{KWFunction}\\
    \mid &\ \text{KWLet}\\
    \mid &\ \text{KWIn}\\
    \mid &\ \text{KWRec}\\
    \mid &\ \text{KWIf}\\
    \mid &\ \text{KWThen}\\
    \mid &\ \text{KWElse}\\
\end{align}
$$

$$
\begin{align}
    \textbf{Literal} ::=&\ \text{NumLiteral}\ \mathbb{N}\\
    \mid &\ \text{BoolLiteral}\ \mathbb{B}\\
\end{align}
$$

In [3]:
sealed trait KW
case object KWFunction extends KW
case object KWLet extends KW
case object KWIn extends KW
case object KWRec extends KW
case object KWIf extends KW
case object KWThen extends KW
case object KWElse extends KW

sealed trait Lit
case class NumLiteral(x : Nat) extends Lit
case class BoolLiteral(x : Bool) extends Lit

sealed trait Lex 
case class Identifier(s : List[Char]) extends Lex
case class Keyword(k : KW) extends Lex
case class Operator(s : List[Char]) extends Lex
case class Literal(l : Lit) extends Lex
case class Comment(s : List[Char]) extends Lex 
case object OpenParen extends Lex
case object CloseParen extends Lex

defined trait KW
defined object KWFunction
defined object KWLet
defined object KWIn
defined object KWRec
defined object KWIf
defined object KWThen
defined object KWElse
defined trait Lit
defined class NumLiteral
defined class BoolLiteral
defined trait Lex
defined class Identifier
defined class Keyword
defined class Operator
defined class Literal
defined class Comment
defined object OpenParen
defined object CloseParen

### Lexing Keywords (5 points)

We will write our lexer by combining a bunch of smaller lexing parsers. Let's start by defining the lexer for keywords.

We will begin by defining separate parsers for each keyword and then combine them with `option`. Below are the definitions for the `let`, `function`, and `rec` keywords.

In [4]:
def pLet1 : Parser[Char, Lex] = bind(string("let"), 
                                    (s : List[Char]) => success(Keyword(KWLet)))

def pFunc1 : Parser[Char, Lex] = bind(string("function"), 
                                    (s : List[Char]) => success(Keyword(KWFunction)))

def pRec1 : Parser[Char, Lex] = bind(string("rec"), 
                                    (s : List[Char]) => success(Keyword(KWRec)))

def pIn1 : Parser[Char, Lex] = bind(string("in"), 
                                    (s : List[Char]) => success(Keyword(KWIn)))

// if, then, and else have a similiar definition

defined function pLet1
defined function pFunc1
defined function pRec1
defined function pIn1

These parser all have a common pattern. This is that we parse the keyword and then discard the results in favor of a constructor that represents it. Because we don't use the result of the `string` part of the parse we can define a handy shorthand function to do this for us. We will call this function `const`.

In [5]:
def const[S,D,E](p : Parser[S,D], d : E) : Parser[S, E] = bind(p, (x : D) => success(d))

defined function const

Note that this function captures the commonality between all of the parsers above, so we can define our parsers in terms of `const` now. We have given you the definition for `let`, fill in the definitions for the rest of the keywords.

In [10]:
def pLet : Parser[Char, Lex] =
    const(string("let"), Keyword(KWLet))

def pFunc : Parser[Char, Lex] =
    const(string("function"), Keyword(KWFunction))

def pRec : Parser[Char, Lex] =
    const(string("rec"), Keyword(KWRec))

def pIn : Parser[Char, Lex] =
   const(string("in"), Keyword(KWIn))

def pIf : Parser[Char, Lex] =
    const(string("if"), Keyword(KWIf))

def pThen : Parser[Char, Lex] =
    const(string("then"), Keyword(KWThen))

def pElse : Parser[Char, Lex] =
    const(string("else"), Keyword(KWElse))

defined function pLet
defined function pFunc
defined function pRec
defined function pIn
defined function pIf
defined function pThen
defined function pElse

In [11]:
def testkw(p: Parser[Char, Lex], w: String, kw: KW) {
    assert(
        runParser(p, w) == Just(Keyword(kw)) && runParser(p, "ssdfsdfdf") == None,
        s"\n\n*** Keyword that failed test: '$w' ***\n"
    )
}
testkw(pLet, "let", KWLet)
testkw(pFunc, "function", KWFunction)
testkw(pRec, "rec", KWRec)
testkw(pIn, "in", KWIn)
testkw(pIf, "if", KWIf)
testkw(pThen, "then", KWThen)
testkw(pElse, "else", KWElse)
passed(3)


*** Tests Passed (3 points) ***


defined function testkw

Now combine these together, using `option`, to create a lexer for all the keywords.

In [12]:
def keywords : Parser[Char, Lex] =
    option(pLet, option(pFunc, option(pRec, option(pIn, option(pIf, 
                                                              option(pThen, pElse))))))

defined function keywords

If you have done this correctly then the parses below should be successful and yield the appropriate keyword:

In [13]:
testkw(keywords, "let", KWLet)
testkw(keywords, "function", KWFunction)
testkw(keywords, "rec", KWRec)
testkw(keywords, "in", KWIn)
testkw(keywords, "if", KWIf)
testkw(keywords, "then", KWThen)
testkw(keywords, "else", KWElse)
passed(2)


*** Tests Passed (2 points) ***


and the parses below should all result in failures

In [14]:
runParser(keywords, "lettuce")
runParser(keywords, "ins")
runParser(keywords, "if then else")
runParser(keywords, "sudo is pronounced sudo")

res13_0: Maybe[Lex] = None
res13_1: Maybe[Lex] = None
res13_2: Maybe[Lex] = None
res13_3: Maybe[Lex] = None

$$
\begin{align}
    \textbf{Lex} ::=&\ \color{Green}{\text{Keyword}\ \textbf{Keyword}}\\
    \mid&\ \text{Literal}\ \textbf{Literal}\\
    \mid &\ \text{Identifier}\ \textbf{String}\\
    \mid &\ \text{Operator}\ \textbf{String}\\
    \mid&\ \text{OpenParen}\\
    \mid&\ \text{CloseParen}
\end{align}
$$

### Lexing Literals (5 points)

#### Booleans

The next part of our lexer to write is the parser for literals. These will include numbers and booleans. The case for booleans is fairly straightforward. We want to check if there is the string `true` or `false` and map these to their constructor in the $\mathbb{B}$ type. You can again use the `const` function to do this(or if you want to write it all out you could also use `bind`)

In [15]:
def pTrue : Parser[Char, Lex] =
    const(string("true"), Literal(BoolLiteral(True)))

def pFalse : Parser[Char, Lex] =
    const(string("false"), Literal(BoolLiteral(False)))


defined function pTrue
defined function pFalse

In [16]:
assert(runParser(pTrue, "true") == Just(Literal(BoolLiteral(True))))
assert(runParser(pTrue, "sffs") == None)
assert(runParser(pFalse, "false") == Just(Literal(BoolLiteral(False))))
assert(runParser(pFalse, "sffs") == None)
passed(2)


*** Tests Passed (2 points) ***


#### Numbers

When we parse numbers we will only want to handle parsing _positive_ numbers. This is because we can represent negative numbers using the constructor $\text{Neg}$ from $\textbf{Expr}$. This will make the job of parsing numbers very easy for use, largely because we have alread defined the `number` parser above. Use `bind` and `number` to parse numbers and return them within a `Literal` constructor from `Lex`.

In [17]:
def pNumber : Parser[Char, Lex] =
    bind(number, (s : Nat) => success(Literal(NumLiteral(s))))

defined function pNumber

In [18]:
def jnls(s: String): Maybe[Lex] = Just(Literal(NumLiteral(stringToNat(string_to_list(s)))))
assert(runParser(pNumber, "10") == jnls("10"))
assert(runParser(pNumber, "142") == jnls("142"))
assert(runParser(pNumber, "abcd") == None)
passed(2)


*** Tests Passed (2 points) ***


defined function jnls

#### The Literal Lexer

We can now combine these, again using `option`, to have a completed literal lexer. Do so below:

In [19]:
def literals : Parser[Char, Lex] =
    option(pTrue, option(pFalse, pNumber))

defined function literals

If you have done all of the above correctly the following parses should be successful:

In [20]:
assert(runParser(literals, "23") == jnls("23"))
assert(runParser(literals, "5") == jnls("5"))
assert(runParser(literals, "0") == jnls("0"))
assert(runParser(literals, "true") == Just(Literal(BoolLiteral(True))))
assert(runParser(literals, "false") == Just(Literal(BoolLiteral(False))))
assert(runParser(literals, "faslse") == None)
assert(runParser(literals, "s123") == None)
passed(1)


*** Tests Passed (1 point) ***


$$
\begin{align}
    \textbf{Lex} ::=&\ \color{Green}{\text{Keyword}\ \textbf{Keyword}}\\
    \mid&\ \color{Green}{\text{Literal}\ \textbf{Literal}}\\
    \mid &\ \text{Identifier}\ \textbf{String}\\
    \mid &\ \text{Operator}\ \textbf{String}\\
    \mid&\ \text{OpenParen}\\
    \mid&\ \text{CloseParen}
\end{align}
$$

### Lexing Identifiers

We'll let identifiers be any combination of letters and numbers, so long as it does not start with a number. We have defined this for you. Read through the definition and try your best to understand it. It will potentially be useful later.

In [21]:
def isAlpha(c : Char) : Bool = if(c >= 'A' & c <= 'z') True else False

def alphanum : Parser[Char, Char] = option(digit, satisfies(isAlpha))

defined function isAlpha
defined function alphanum

In [22]:
def identifier : Parser[Char, Lex] = bind(satisfies(isAlpha), 
                                          (x : Char) => bind(many(alphanum),
                                          (xs : List[Char]) => success(Identifier(Cons(x,xs)))))

defined function identifier

$$
\begin{align}
    \textbf{Lex} ::=&\ \color{Green}{\text{Keyword}\ \textbf{Keyword}}\\
    \mid&\ \color{Green}{\text{Literal}\ \textbf{Literal}}\\
    \mid &\ \color{Green}{\text{Identifier}\ \textbf{String}}\\
    \mid &\ \text{Operator}\ \textbf{String}\\
    \mid&\ \text{OpenParen}\\
    \mid&\ \text{CloseParen}
\end{align}
$$

### Lexing Operators (5 points)

We will give a the set of operators for Lettuce as any non-alphabetic symbol we use. This will be: 

`+`, `-`, `*`, `==`, `&`, `|` and `=`

Fill in the missing section of the operators lexer below. The `satisfies` function above will prove very useful:

In [23]:
def isOperator(x : Char) : Bool = x match {
    case '+' => True
    case '-' => True
    case '*' => True
    case '=' => True
    case '&' => True
    case '|' => True
    case _   => False
}

val MISSING = satisfies(isOperator)
    
def operators : Parser[Char, Lex] = option(
    bind(string("=="), (str : List[Char]) => success(Operator(str))),
    bind(MISSING     , (c : Char) => success(Operator(singleton(c))))
)

defined function isOperator
MISSING: List[Char] => List[(Char, List[Char])] = ammonite.$sess.cmd1$Helper$$Lambda$3203/1209878496@67facc95
defined function operators

If this is defined correctly the following parses should be successful:

In [24]:
assert(runParser(operators, "*") == Just(Operator(Cons('*', Empty))))
assert(runParser(operators, "+") == Just(Operator(Cons('+', Empty))))
assert(runParser(operators, "-") == Just(Operator(Cons('-', Empty))))
assert(runParser(operators, "=") == Just(Operator(Cons('=', Empty))))
assert(runParser(operators, "==") == Just(Operator(Cons('=', Cons('=', Empty)))))
assert(runParser(operators, "faslse") == None)
assert(runParser(operators, "s123") == None)
passed(5)


*** Tests Passed (5 points) ***


$$
\begin{align}
    \textbf{Lex} ::=&\ \color{Green}{\text{Keyword}\ \textbf{Keyword}}\\
    \mid&\ \color{Green}{\text{Literal}\ \textbf{Literal}}\\
    \mid &\ \color{Green}{\text{Identifier}\ \textbf{String}}\\
    \mid &\ \color{Green}{\text{Operator}\ \textbf{String}}\\
    \mid&\ \text{OpenParen}\\
    \mid&\ \text{CloseParen}
\end{align}
$$

### Parenthesis (5 points)

Again use the `const` and `option` functions to create a lexer for open and closing parenthesis.

In [ ]:
def parens : Parser[Char, Lex] =
   option(const(string("("), OpenParen), const(string(")"), CloseParen))

If written properly, the following parser should output a list of lexes corresponding to parenthesis:

In [ ]:
assert(runParser(parens, "(") == Just(OpenParen))
assert(runParser(parens, "sdf") == None)
passed(2)

In [ ]:
assert(runParser(parens, ")") == Just(CloseParen))
assert(runParser(parens, "sdf") == None)
assert(runParser(many(parens), "()())((()()))") == Just(Cons(OpenParen,Cons(CloseParen,Cons(OpenParen,Cons(CloseParen,Cons(CloseParen,Cons(OpenParen,Cons(OpenParen,Cons(OpenParen,Cons(CloseParen,Cons(OpenParen,Cons(CloseParen,Cons(CloseParen,Cons(CloseParen,Empty)))))))))))))))
passed(3)

$$
\begin{align}
    \textbf{Lex} ::=&\ \color{Green}{\text{Keyword}\ \textbf{Keyword}}\\
    \mid&\ \color{Green}{\text{Literal}\ \textbf{Literal}}\\
    \mid &\ \color{Green}{\text{Identifier}\ \textbf{String}}\\
    \mid &\ \color{Green}{\text{Operator}\ \textbf{String}}\\
    \mid&\ \color{Green}{\text{OpenParen}}\\
    \mid&\ \color{Green}{\text{CloseParen}}
\end{align}
$$

### Putting the Lexer together

Below we define some functions to bring everything together. Try your best to understand what is happening at a high level. Most of these functions have special behavior for handling parenthesis and whitespace.

In [ ]:
def lexOne : Parser[Char, Lex] = option(keywords, 
                                 option(operators, 
                                 option(parens,
                                 option(literals,
                                        identifier))))

def lexWhitespace : Parser[Char, List[Lex]] = const(whitespace, Empty)

def lexParens : Parser[Char, List[Lex]] = bind(parens, (x : Lex) => 
                                          bind(maybeWhitespace,
                                               (ws : List[Char]) => success(singleton(x))))


def lexToken : Parser[Char, List[Lex]] = bind(lexOne, 
                                             (l : Lex) => bind(option(lexParens, lexWhitespace),
                                             (sep : List[Lex]) => success(Cons(l, sep))))


In [ ]:
def concat[A](xs : List[List[A]]) : List[A] = xs match {
    case Empty => Empty
    case Cons(xs, xxs) => append(xs, concat(xxs))
}


def lexer : Parser[Char, List[Lex]] = bind(many(lexToken), 
                   (ls : List[List[Lex]]) =>
                   success(concat(ls)))

If everything in this notebook is done correctly then the following strings should successfully lex. Note that they always will need to end in a trailing whitespace in order to parse correctly. (This was the bug I mentioned in lecture, I never could quite squish it)

In [ ]:
runParser(lexer, "let x = 5 in x + x ")

In [ ]:
runParser(lexer, "let f = function(x) x + x in f(5) ")

In [ ]:
runParser(lexer, "5 + 5 ")